In [1]:
from pathlib import Path

from foundata import cmap, ltds, nhts, nts, qhts, vista

In [2]:
configs_root = Path("configs")

In [3]:
# cmap
hh_config = configs_root / "cmap" / "hh_dictionary.yaml"
person_config = configs_root / "cmap" / "person_dictionary.yaml"
trip_config = configs_root / "cmap" / "trip_dictionary.yaml"

data_root = Path("/home/fred/Data/foundata/CMAP")

hhs = cmap.load_households(data_root, hh_config)
hhs.head()

TypeError: unexpected value while building Series of type String; found value of type List(Int64): [0, 15000]

Hint: Try setting `strict=False` to allow passing data with mixed types.